# Proyecto Open Data I
## Radares, y su eficiencia en la CAM
### Recopilación, limpieza y tratamiento de los datos
Este cuaderno pretende enseñar el proceso de limpieza de los datos relativos a los radares en la CAM
_Paula Gómez Lucas, Alejandro Majado Martínez_

In [1]:
# Importar librerías
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

from pandas_summary import DataFrameSummary

from scipy.stats import chisquare
import numpy as np

A continuación, se muestra la clase que está compuesta de todos los métodos que se encargan de la limpieza y transformación de los datos

In [2]:
class CSVDataLoader:
    """
    A class for loading and cleaning CSV data from a specified folder path.

    Attributes:
    -----------
    folder_path : str
        The path to the folder containing the CSV files to be loaded.

    data : dict
        A dictionary containing the loaded CSV data, where the keys are the file names and the values are the corresponding dataframes.
    """

    def __init__(self, folder_path):
        """
        Initializes a CSVDataLoader object with the specified folder path.

        Parameters:
        -----------
        folder_path : str
            The path to the folder containing the CSV files to be loaded.
        """
        self.folder_path = folder_path
        self.data = {}
        self.filename = []
        self.keys = []

    def load_data(self):
        """
        Loads CSV data from the specified folder path into a dictionary.

        Returns:
        --------
        None
        """
        csv_files = [f for f in os.listdir(self.folder_path) if f.endswith('.csv')]
        folders = ("datasets/actuacionesBomberos", "datasets/estaciones", "datasets/accidentalidad")
        for folder in folders:
            df = None
            for file in os.listdir(folder):
                filepath = folder + "/" + file
                df1 = pd.read_csv(filepath, sep=';', encoding='utf-8', low_memory=False)
                df = pd.concat([df, df1])
            self.data[str(folder)] = df

        for file_name in csv_files:
            file_path = os.path.join(self.folder_path, file_name)
            try:
                df = pd.read_csv(file_path, sep=';', encoding='latin-1', low_memory=False)
                self.data[str(file_name)] = df
                self.filename.append(file_name)
            except Exception as e:
                print(f"Error al leer {file_name}: {str(e)}")

        for value in self.data.keys():
            self.keys.append(value)

    def clean_data(self):
        """
        Cleans the loaded CSV data by renaming columns, removing whitespace, dropping null values and duplicates, and converting date columns to datetime format.

        Returns:
        --------
        None
        """
        columna_borrar = "Unnamed"
        for df in self.data:
            for j in self.data[df].columns:
                if columna_borrar in j:
                    while j in self.data[df].columns:
                        self.data[df] = self.data[df].drop(j, axis=1)
                        self.data[df] = self.data[df].dropna(how='all', axis=0)
                        
            self.data[df] = self.data[df].rename(columns = lambda x: x.strip().lower().replace(' ', '_'))
            self.data[df] = self.data[df].map(lambda x: x.strip() if isinstance(x, str) else x)
            self.data[df] = self.data[df].dropna(how='all', axis=0)
            self.data[df] = self.data[df].drop_duplicates()
            self.data[df] = self.data[df].loc[:, ~self.data[df].columns.duplicated()]
            self.data[df].columns = map(str.upper, self.data[df].columns)

            if 'FECHA' in self.data[df].columns:
                self.data[df]['FECHA'] = pd.to_datetime(self.data[df]['FECHA'], format='%d/%m/%Y')

#           num_cols = self.data[i].select_dtypes(include='number').columns
#           for col in num_cols:
#               self.data[i][col] = self.data[i][col].fillna(self.data[i][col].mean())

    def get_info(self, filename):
        print(self.data[filename].isnull().sum())
        print(self.data[filename].info())
        
    def get_nan_columns(self):
        j = 0
        for i in self.data:
            print(self.keys[j])
            self.get_info(i)
            j+=1
      
    def get_cleaned_data(self):
        """
        Returns the cleaned CSV data as a dictionary.

        Returns:
        --------
        dict
            A dictionary containing the cleaned CSV data, where the keys are the file names and the values are the corresponding dataframes.
        """
        return self.data

    def create_graph(df, colummn, name):
        frec = df[''+str(colummn)].value_counts()
        aux_df = pd.DataFrame(frec)
        aux_df.columns = ["Frecuencia absoluta"]
        aux_df["Frecuencia relativa"] = 100*aux_df["Frecuencia absoluta"] / len(df)
        frec_rel_cumsum = aux_df["Frecuencia relativa"].cumsum()
        aux_df["Frecuencia relativa acumulada"] = frec_rel_cumsum
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.set_title('Distribución de '+ str(name))
        ax.bar(aux_df.index, aux_df['Frecuencia absoluta'], color='blue')
        ax2 = ax.twinx()
        ax2.plot(aux_df.index, aux_df['Frecuencia relativa acumulada'], color='red', marker='o', ms = 5)
        ax2.yaxis.set_major_formatter(PercentFormatter())
        ax.tick_params(axis='y', color = 'blue')
        ax2.tick_params(axis='y', color = 'red')
        ax.set_xticklabels(aux_df.index, rotation=90)
        plt.show()

    def dataframe_summary(self, filename):

        numeric_mask = self.data[filename].select_dtypes(include='number').columns

        # Create a DataFrameSummary object
        summary = DataFrameSummary(self.data[filename])
    
        # Display summary statistics
        summary_stats = summary.summary()
    
        # Display basic statistics using describe()
        summary_stats = self.data[filename][numeric_mask].describe(include='all')

        # Plot histograms for all numeric columns
        self.data[filename][numeric_mask].hist(bins=20, figsize=(15, 10))
        plt.suptitle('Histograms for Numeric Columns')
        plt.show()
    
        # Plot a correlation matrix heatmap for numeric columns
        if len(self.data[filename].select_dtypes(numeric_mask).columns) > 1:
            corr_matrix = self.data[filename].corr()
            plt.figure(figsize=(12, 8))
            sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
            plt.title('Correlation Matrix Heatmap')
            plt.show()
        else:
            print("Not enough numeric columns to generate a correlation matrix heatmap.")
    
        return summary_stats

Una vez está definida la clase con sus métodos, procedemos a declarar las variables que nos permiten trabajar con ello

In [3]:
folder_path = "datasets"
data_loader = CSVDataLoader(folder_path)

La siguiente función carga los datos de los csv a los dataframes

In [4]:
data_loader.load_data()

Limpiamos los datos eliminando las columnas autogeneradas con NaNs, renombramos las columnas para que sean uniformes (minúsculas y con barra bajas), eliminamos las filas de NaNs, eliminamos las filas duplicadas, formateamos todas las variables fecha para que sean consistentes (dd/mm/aaaa), sustituimos los NaNs de las variables numéricas con la media correspondiente a su variable.

In [5]:
data_loader.clean_data()
data = data_loader.get_cleaned_data()

Por último, para confirmar que los datos se han cargado bien, utilizamos el método get_nan_columns para ver cuántos datos en cada columna quedan nulos, así como usamos el método info() de pandas para ver un resumen de todas las columnas y comprobamos también que todo el formateo de las columnas se ha realizado sin problema.

In [6]:
data_loader.get_nan_columns()

datasets/actuacionesBomberos
AÑO                         0
MES                         0
DISTRITO                    0
FUEGOS                      0
DAÑOS_EN_CONSTRUCCION       0
SALVAMENTOS_Y_RESCATES      0
DAÑOS_POR_AGUA              0
INCIDENTES_DIVERSOS         0
SALIDAS_SIN_INTERVENCION    0
SERVICIOS_VARIOS            0
TOTAL                       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 439 entries, 0 to 262
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   AÑO                       439 non-null    int64 
 1   MES                       439 non-null    object
 2   DISTRITO                  439 non-null    object
 3   FUEGOS                    439 non-null    int64 
 4   DAÑOS_EN_CONSTRUCCION     439 non-null    int64 
 5   SALVAMENTOS_Y_RESCATES    439 non-null    int64 
 6   DAÑOS_POR_AGUA            439 non-null    int64 
 7   INCIDENTES_DIVERSOS       439 non-nul

Efectivamente, se realiza sin problemas. El siguiente paso es el análisis de los datasets columna a columna para revisar qué método usar para rellenar los datos faltantes dependiendo de cada atributo.  

### Sustitución de valores faltantes
Los datasets actuaciones bomberos y estaciones no tienen datos faltantes, por lo que sólo nos queda trabajar con los otros 8 datasets, que podemos observar aquí:

In [7]:
for i in data.keys():
    print(i)

datasets/actuacionesBomberos
datasets/estaciones
datasets/accidentalidad
DireccionesVigentes_20231004.csv
DireccionesEvolucionHistorica_20231004.csv
VialesVigentesDistritosBarrios_20231004.csv
RADARES_FIJOS_vDTT.csv
VialesEvolucionHistorica_20231004.csv
VialesVigentesDistritos_20231004.csv
VialesVigentes_20231004.csv


Empecemos con accidentalidad: 

In [8]:
data_loader.get_info('datasets/accidentalidad')
data_loader.data['datasets/accidentalidad']

NUM_EXPEDIENTE               0
FECHA                        0
HORA                         0
LOCALIZACION                 0
NUMERO                       6
COD_DISTRITO                 6
DISTRITO                     6
TIPO_ACCIDENTE               5
ESTADO_METEOROLÓGICO     20120
TIPO_VEHICULO              816
TIPO_PERSONA                 3
RANGO_EDAD                   0
SEXO                         0
COD_LESIVIDAD            85278
LESIVIDAD                85278
COORDENADA_X_UTM             3
COORDENADA_Y_UTM             3
POSITIVA_ALCOHOL           718
POSITIVA_DROGA          195001
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 51810
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   NUM_EXPEDIENTE        195610 non-null  object        
 1   FECHA                 195610 non-null  datetime64[ns]
 2   HORA                  195610 non-null  object       

,NUM_EXPEDIENTE,FECHA,HORA,LOCALIZACION,NUMERO,COD_DISTRITO,DISTRITO,TIPO_ACCIDENTE,ESTADO_METEOROLÓGICO,TIPO_VEHICULO,TIPO_PERSONA,RANGO_EDAD,SEXO,COD_LESIVIDAD,LESIVIDAD,COORDENADA_X_UTM,COORDENADA_Y_UTM,POSITIVA_ALCOHOL,POSITIVA_DROGA
0,2023S000001,2023-01-01,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Conductor,De 55 a 59 años,Mujer,14.0,Sin asistencia sanitaria,443397.166,4478129.388,N,NaN
1,2023S000001,2023-01-01,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Pasajero,De 21 a 24 años,Hombre,14.0,Sin asistencia sanitaria,443397.166,4478129.388,N,NaN
3,2023S000001,2023-01-01,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Pasajero,De 21 a 24 años,Mujer,7.0,Asistencia sanitaria sólo en el lugar del acci...,443397.166,4478129.388,N,NaN
4,2023S000001,2023-01-01,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Todo terreno,Pasajero,De 55 a 59 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,443397.166,4478129.388,N,NaN
5,2023S000001,2023-01-01,1:15:00,"AVDA. ALFONSO XIII, 33",33,5.0,CHAMARTÍN,Alcance,Despejado,Turismo,Conductor,De 45 a 49 años,Mujer,14.0,Sin asistencia sanitaria,443397.166,4478129.388,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51806,2019S039993,2019-11-10,11:00:00,PLAZA. ELIPTICA / CALL. MARCELO USERA,1,12.0,USERA,Colisión lateral,Despejado,Turismo,Pasajero,De 60 a 64 años,Mujer,14.0,Sin asistencia sanitaria,"439115,482","4470746,354",N,NaN
51807,2019S040001,2019-03-13,12:47:00,"CALL. ALFONSO FERNANDEZ CLAUSELLS, 7",7,9.0,MONCLOA-ARAVACA,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,Desconocido,Desconocido,NaN,NaN,"438541,541","4479292,853",N,NaN
51808,2019S040001,2019-03-13,12:47:00,"CALL. ALFONSO FERNANDEZ CLAUSELLS, 7",7,9.0,MONCLOA-ARAVACA,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,Más de 74 años,Mujer,NaN,NaN,"438541,541","4479292,853",N,NaN
51809,2019S040007,2019-11-03,10:10:00,"CALL. HALCONERO DEL REY, 0",0,11.0,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 25 a 29 años,Mujer,NaN,NaN,"438343,989","4468673,466",N,NaN


Hay algunos datos faltantes que tiene sentido que lo sean, y podemos sustituir por un buzzword de algún tipo que nos haga saber que se trate de esto, como lo es que en un accidente en el que no se han producido lesiones, la lesividad sea nula y tampoco haya código de la misma, y como ambas cifras coinciden, es lógico pensar que se trata de las mismas situaciones. Podemos sustituir entonces todos los NaNs de Lesividad faltantes por 'Sin asistencia sanitaria' y el código por -1.0.

In [9]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'] = data_loader.data['datasets/accidentalidad']['LESIVIDAD'].fillna('Sin asistencia sanitaria')
data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'] = data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].fillna(-1.0)

Por otro lado, positivo en droga tiene valor sólo si daba positivo, por lo que rellenar los valores faltantes con 0 es lo más lógico (siendo 0 negativo en droga). 

In [10]:
data_loader.data['datasets/accidentalidad']['POSITIVA_DROGA'] = data_loader.data['datasets/accidentalidad']['POSITIVA_DROGA'].fillna(0)

Número, código de distrito, tipo de accidente, coordenadas (x e y), son atributos a los que sólo les falta un dato cada uno, por lo que no es representativo esta falta de datos y podemos rellenarlos con el valor más habitual. 

In [11]:
numero = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
codDistrito = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
distrito = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
accidente = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
coorX = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
coorY = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]


data_loader.data['datasets/accidentalidad']['NUMERO'] = data_loader.data['datasets/accidentalidad']['NUMERO'].fillna(numero)
data_loader.data['datasets/accidentalidad']['COD_DISTRITO'] = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].fillna(codDistrito)
data_loader.data['datasets/accidentalidad']['DISTRITO'] = data_loader.data['datasets/accidentalidad']['DISTRITO'].fillna(distrito)
data_loader.data['datasets/accidentalidad']['TIPO_ACCIDENTE'] = data_loader.data['datasets/accidentalidad']['TIPO_ACCIDENTE'].fillna(accidente)
data_loader.data['datasets/accidentalidad']['COORDENADA_X_UTM'] = data_loader.data['datasets/accidentalidad']['COORDENADA_X_UTM'].fillna(coorX)
data_loader.data['datasets/accidentalidad']['COORDENADA_Y_UTM'] = data_loader.data['datasets/accidentalidad']['COORDENADA_Y_UTM'].fillna(coorY)


Por último, donde queda dilema es en positivo alcohol, tipo de vehículo y estado meteorológico. En esta situación, lo más apropiado es ver si, relacionando estos atributos con algún otro, es más probable que los atributos valgan uno u otro valor.

- Estado meteorológico. Hay 7 valores posibles: despejado, lluvia débil, lluvia intensa, granizando, nevando, nublado, se desconoce. Aquí, por lo tanto, hay 3 vías de actuación:
    - Rellenar con "se desconoce", i.e.: ser fieles a lo que se sabe, reducir la proporción de datos artificiales (hay un 11% de datos faltantes), solución sencilla.
    - Rellenar con el valor más frecuente: despejado (representa el 75% de los datos), i.e.: solución con datos artificiales más sencilla.
    - Rellenar con valores aleatorios según la proporción en la que aparecen los datos, i.e.: el 75% de los datos faltantes se rellenan arbitrariamente con "Despejado".

    Lo que mejor preserva los datos es, rellenar con "se desconoce", pues la variable existe previamente.

In [12]:
data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'] = data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'].fillna('Se desconoce')

- Tipo de vehículo. Hay 34 valores posibles: Ambulancia SAMUR, autobús EMT, autobús, autobús articulado, autobús articulado EMT, autocaravana, bicicleta, bicicleta EPAC (pedaleo asistido), camión de bomberos, camión rígido, ciclo, ciclomotor, ciclomotor de dos ruedas L1e-B, cuadriciclo ligero, cuadriciclo no ligero, furgoneta, maquinaria de obras, microbús <= 17 plazas, moto de tres ruedas > 125cc, moto de tres ruedas hasta 125cc, motocicleta > 125cc, motocicleta hasta 125cc, otros vehículos con motor, otros vehículos sin motor, patinete no eléctrico, remolque, semirremolque, sin especificar, todo terreno, tractocamión, tren/metro, turismo (68%), VMU eléctrico, vehículo articulado. En esta variable hay 0.6% de valores faltantes, lo cual no es significativo, i.e.: la sustitución que elijamos tendrá menos repercusión en el estudio final. Aquí, por lo tanto, hay 2 vías de actuación:
    - Rellenar con "sin especificar", i.e.: solución sencilla y descriptiva pero que puede dar lugar a interpretaciones erróneas, pues puede haber sido otro tipo de vehículo que no se había registrado.
    - Rellenar con el valor más probable según otro atributo (por ejemplo, código de lesividad).
    
    La solución más apropiada es rellenar con el valor más probable según código de lesividad, por lo que vamos a ver primero cómo se relacionan ambos atributos y después rellenaremos los valores faltantes con el valor más probable.

In [13]:
data_loader.data['datasets/accidentalidad']['TIPO_VEHICULO'] = data_loader.data['datasets/accidentalidad']['TIPO_VEHICULO'].fillna(data_loader.data['datasets/accidentalidad'].groupby('COD_LESIVIDAD')['TIPO_VEHICULO'].transform(lambda x:x.mode().iat[0]))

Para rellenar los datos de positivo en alcohol, usaremos lo más habitual según grupo de edad, sexo y lesividad de la persona

In [14]:
data_loader.data['datasets/accidentalidad']["POSITIVA_ALCOHOL"] = data_loader.data['datasets/accidentalidad'].groupby(['RANGO_EDAD','SEXO','LESIVIDAD'])['POSITIVA_ALCOHOL'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

Para determinar el tipo de persona, usaremos de criterio el rango de edad y la lesividad -si bien sólo faltan 3 datos, en esta situación podemos realizar un ajuste más específico-

In [15]:
data_loader.data['datasets/accidentalidad']["TIPO_PERSONA"] = data_loader.data['datasets/accidentalidad'].groupby(['RANGO_EDAD','LESIVIDAD'])['TIPO_PERSONA'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

In [16]:
data_loader.get_info('datasets/accidentalidad')

NUM_EXPEDIENTE          0
FECHA                   0
HORA                    0
LOCALIZACION            0
NUMERO                  0
COD_DISTRITO            0
DISTRITO                0
TIPO_ACCIDENTE          0
ESTADO_METEOROLÓGICO    0
TIPO_VEHICULO           0
TIPO_PERSONA            0
RANGO_EDAD              0
SEXO                    0
COD_LESIVIDAD           0
LESIVIDAD               0
COORDENADA_X_UTM        0
COORDENADA_Y_UTM        0
POSITIVA_ALCOHOL        0
POSITIVA_DROGA          0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 195610 entries, 0 to 51810
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   NUM_EXPEDIENTE        195610 non-null  object        
 1   FECHA                 195610 non-null  datetime64[ns]
 2   HORA                  195610 non-null  object        
 3   LOCALIZACION          195610 non-null  object        
 4   NUMERO                195610 

Como podemos comprobar, no quedan datos faltantes. Pasamos a Actuaciones Bomberos:

In [17]:
data_loader.get_info('datasets/actuacionesBomberos')

AÑO                         0
MES                         0
DISTRITO                    0
FUEGOS                      0
DAÑOS_EN_CONSTRUCCION       0
SALVAMENTOS_Y_RESCATES      0
DAÑOS_POR_AGUA              0
INCIDENTES_DIVERSOS         0
SALIDAS_SIN_INTERVENCION    0
SERVICIOS_VARIOS            0
TOTAL                       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 439 entries, 0 to 262
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   AÑO                       439 non-null    int64 
 1   MES                       439 non-null    object
 2   DISTRITO                  439 non-null    object
 3   FUEGOS                    439 non-null    int64 
 4   DAÑOS_EN_CONSTRUCCION     439 non-null    int64 
 5   SALVAMENTOS_Y_RESCATES    439 non-null    int64 
 6   DAÑOS_POR_AGUA            439 non-null    int64 
 7   INCIDENTES_DIVERSOS       439 non-null    int64 
 8   SALIDAS_SIN_

Este dataset no tiene datos faltantes, pasamos al siguiente. Estaciones:

In [18]:
data_loader.get_info('datasets/estaciones')
data_loader.data['datasets/estaciones']

FDIA     0
FEST     0
FSEN     0
HOR1     0
HOR2     0
HOR3     0
HOR4     0
HOR5     0
HOR6     0
HOR7     0
HOR8     0
HOR9     0
HOR10    0
HOR11    0
HOR12    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 86140 entries, 0 to 7315
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   FDIA    86140 non-null  object 
 1   FEST    86140 non-null  object 
 2   FSEN    86140 non-null  object 
 3   HOR1    86140 non-null  float64
 4   HOR2    86140 non-null  float64
 5   HOR3    86140 non-null  float64
 6   HOR4    86140 non-null  float64
 7   HOR5    86140 non-null  float64
 8   HOR6    86140 non-null  float64
 9   HOR7    86140 non-null  float64
 10  HOR8    86140 non-null  float64
 11  HOR9    86140 non-null  float64
 12  HOR10   86140 non-null  float64
 13  HOR11   86140 non-null  float64
 14  HOR12   86140 non-null  float64
dtypes: float64(12), object(3)
memory usage: 10.5+ MB
None


,FDIA,FEST,FSEN,HOR1,HOR2,HOR3,HOR4,HOR5,HOR6,HOR7,HOR8,HOR9,HOR10,HOR11,HOR12
0,01/10/2022,ES01,1-,940.0,811.0,472.0,375.0,358.0,405.0,347.0,485.0,676.0,824.0,1048.0,1173.0
1,01/10/2022,ES01,1=,1496.0,1544.0,944.0,928.0,1112.0,1176.0,1357.0,1701.0,1589.0,1077.0,967.0,1095.0
2,01/10/2022,ES01,2-,675.0,538.0,430.0,378.0,331.0,461.0,283.0,307.0,610.0,762.0,1143.0,1212.0
3,01/10/2022,ES01,2=,1267.0,1114.0,861.0,963.0,1109.0,1139.0,1104.0,1211.0,1168.0,952.0,808.0,803.0
4,01/10/2022,ES02,1-,385.0,308.0,290.0,252.0,211.0,198.0,186.0,189.0,230.0,313.0,391.0,472.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7311,31/07/2023,ES59,2=,164.0,154.0,129.0,143.0,161.0,187.0,190.0,140.0,108.0,102.0,62.0,34.0
7312,31/07/2023,ES60,1-,45.0,26.0,25.0,22.0,42.0,138.0,331.0,413.0,409.0,402.0,493.0,419.0
7313,31/07/2023,ES60,1=,363.0,306.0,277.0,272.0,347.0,374.0,394.0,323.0,240.0,130.0,102.0,64.0
7314,31/07/2023,ES60,2-,67.0,45.0,26.0,35.0,22.0,85.0,192.0,253.0,342.0,357.0,395.0,400.0


Este dataset tampoco tiene datos faltantes. Apreciamos que FSEN, aparte de indicar el sentido del tráfico, también indica si son las primeras 12h del día (1- y 2-), o las últimas (1=, 2=). Hacemos 12 nuevos atributos para suplir esto en un formato 24h que es más cómodo.

In [19]:
# Convert 'date' column to datetime if it's not already
data_loader.data['datasets/estaciones']['FDIA'] = pd.to_datetime(data_loader.data['datasets/estaciones']['FDIA'], format='mixed', dayfirst=True)

In [20]:
data_loader.get_info('datasets/estaciones')

FDIA     0
FEST     0
FSEN     0
HOR1     0
HOR2     0
HOR3     0
HOR4     0
HOR5     0
HOR6     0
HOR7     0
HOR8     0
HOR9     0
HOR10    0
HOR11    0
HOR12    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 86140 entries, 0 to 7315
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   FDIA    86140 non-null  datetime64[ns]
 1   FEST    86140 non-null  object        
 2   FSEN    86140 non-null  object        
 3   HOR1    86140 non-null  float64       
 4   HOR2    86140 non-null  float64       
 5   HOR3    86140 non-null  float64       
 6   HOR4    86140 non-null  float64       
 7   HOR5    86140 non-null  float64       
 8   HOR6    86140 non-null  float64       
 9   HOR7    86140 non-null  float64       
 10  HOR8    86140 non-null  float64       
 11  HOR9    86140 non-null  float64       
 12  HOR10   86140 non-null  float64       
 13  HOR11   86140 non-null  float64       
 14  HOR12   86

In [21]:
df = data_loader.data['datasets/estaciones']
# Split the FSEN values into direction and hour parts
df[['FSEN_DIRECTION', 'FSEN_AMPM']] = df['FSEN'].str.extract(r'(\d)([=-])')

# Drop unnecessary columns
df.drop(['FSEN'], axis=1, inplace=True)

index = ['FDIA', 'FEST', 'FSEN_DIRECTION', 'FSEN_AMPM', 'HOR1', 'HOR2', 'HOR3', 'HOR4', 'HOR5', 'HOR6', 'HOR7', 'HOR8', 'HOR9', 'HOR10', 'HOR11', 'HOR12', 'HOR13', 'HOR14', 'HOR15', 'HOR16', 'HOR17', 'HOR18', 'HOR19', 'HOR20', 'HOR21', 'HOR22', 'HOR23', 'HOR24']

df.reindex(columns=index)

,FDIA,FEST,FSEN_DIRECTION,FSEN_AMPM,HOR1,HOR2,HOR3,HOR4,HOR5,HOR6,...,HOR15,HOR16,HOR17,HOR18,HOR19,HOR20,HOR21,HOR22,HOR23,HOR24
0,2022-10-01,ES01,1,-,940.0,811.0,472.0,375.0,358.0,405.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-01,ES01,1,=,1496.0,1544.0,944.0,928.0,1112.0,1176.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-01,ES01,2,-,675.0,538.0,430.0,378.0,331.0,461.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-01,ES01,2,=,1267.0,1114.0,861.0,963.0,1109.0,1139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-10-01,ES02,1,-,385.0,308.0,290.0,252.0,211.0,198.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7311,2023-07-31,ES59,2,=,164.0,154.0,129.0,143.0,161.0,187.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7312,2023-07-31,ES60,1,-,45.0,26.0,25.0,22.0,42.0,138.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7313,2023-07-31,ES60,1,=,363.0,306.0,277.0,272.0,347.0,374.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7314,2023-07-31,ES60,2,-,67.0,45.0,26.0,35.0,22.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
def group_fn(dff: pd.DataFrame):
    # dff contiene 2 rows de am y pm
    am = dff.loc[dff['FSEN_AMPM'] == "-"].iloc[0]
    pm = dff.loc[dff['FSEN_AMPM'] == "="].iloc[0]
    par = am[['HOR1', 'HOR2', 'HOR3', 'HOR4', 'HOR5', 'HOR6', 'HOR7', 'HOR8', 'HOR9', 'HOR10', 'HOR11', 'HOR12']].copy()
    
    par['HOR13'] = pm['HOR1']
    par['HOR14'] = pm['HOR2']
    par['HOR15'] = pm['HOR3']
    par['HOR16'] = pm['HOR4']
    par['HOR17'] = pm['HOR5']
    par['HOR18'] = pm['HOR6']
    par['HOR19'] = pm['HOR7']
    par['HOR20'] = pm['HOR8']
    par['HOR21'] = pm['HOR9']
    par['HOR22'] = pm['HOR10']
    par['HOR23'] = pm['HOR11']
    par['HOR24'] = pm['HOR12']
    return par
df = data_loader.data['datasets/estaciones']
df = df.groupby(['FEST', 'FDIA', 'FSEN_DIRECTION']).apply(group_fn)
df

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43070 entries, ('ES01', Timestamp('2022-08-01 00:00:00'), '1') to ('ES60', Timestamp('2023-07-31 00:00:00'), '2')
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   HOR1    43070 non-null  float64
 1   HOR2    43070 non-null  float64
 2   HOR3    43070 non-null  float64
 3   HOR4    43070 non-null  float64
 4   HOR5    43070 non-null  float64
 5   HOR6    43070 non-null  float64
 6   HOR7    43070 non-null  float64
 7   HOR8    43070 non-null  float64
 8   HOR9    43070 non-null  float64
 9   HOR10   43070 non-null  float64
 10  HOR11   43070 non-null  float64
 11  HOR12   43070 non-null  float64
 12  HOR13   43070 non-null  float64
 13  HOR14   43070 non-null  float64
 14  HOR15   43070 non-null  float64
 15  HOR16   43070 non-null  float64
 16  HOR17   43070 non-null  float64
 17  HOR18   43070 non-null  float64
 18  HOR19   43070 non-null  float64
 19  HOR20   43070 non-nul

In [27]:
df = data_loader.data['datasets/estaciones']
df = df.groupby(['FEST', 'FDIA', 'FSEN_DIRECTION']).apply(group_fn)
df

HOR1      183.0
HOR2      128.0
HOR3      104.0
HOR4       78.0
HOR5      164.0
HOR6      593.0
HOR7     1379.0
HOR8     1483.0
HOR9     1384.0
HOR10    1226.0
HOR11    1240.0
HOR12    1284.0
HOR13    1220.0
HOR14    1445.0
HOR15    1232.0
HOR16     945.0
HOR17     993.0
HOR18    1109.0
HOR19     924.0
HOR20     944.0
HOR21     677.0
HOR22     556.0
HOR23     552.0
HOR24     386.0
Name: 0, dtype: object
HOR1      205.0
HOR2      155.0
HOR3      108.0
HOR4       78.0
HOR5      116.0
HOR6      236.0
HOR7      528.0
HOR8      948.0
HOR9      918.0
HOR10     915.0
HOR11     942.0
HOR12    1022.0
HOR13     940.0
HOR14    1002.0
HOR15    1028.0
HOR16     891.0
HOR17     940.0
HOR18     957.0
HOR19     849.0
HOR20     790.0
HOR21     572.0
HOR22     487.0
HOR23     348.0
HOR24     275.0
Name: 2, dtype: object
HOR1      221.0
HOR2      111.0
HOR3       85.0
HOR4       77.0
HOR5      166.0
HOR6      506.0
HOR7     1414.0
HOR8     1513.0
HOR9     1335.0
HOR10    1263.0
HOR11    1287.0
HOR12    1

KeyboardInterrupt: 

In [ ]:
df11 = df
df11[df11['FSEN_AMPM'] == "="].rename(columns={'HOR1': 'HOR13'})


,FDIA,FEST,HOR13,HOR2,HOR3,HOR4,HOR5,HOR6,HOR7,HOR8,HOR9,HOR10,HOR11,HOR12,FSEN_DIRECTION,FSEN_AMPM
1,2023-04-01,ES01,1223.0,1165.0,832.0,884.0,1016.0,1058.0,1104.0,1407.0,1276.0,994.0,986.0,965.0,1,=
3,2023-04-01,ES01,1206.0,1077.0,748.0,923.0,964.0,1104.0,1159.0,1083.0,971.0,804.0,767.0,709.0,2,=
5,2023-04-01,ES02,495.0,483.0,466.0,397.0,397.0,472.0,517.0,542.0,614.0,514.0,467.0,400.0,1,=
7,2023-04-01,ES02,1048.0,1042.0,1074.0,948.0,919.0,1034.0,977.0,1031.0,1036.0,1125.0,1125.0,1024.0,2,=
9,2023-04-01,ES03,1147.0,1260.0,1214.0,810.0,750.0,918.0,1022.0,1030.0,1029.0,1051.0,710.0,549.0,1,=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7071,2022-09-30,ES58,281.0,380.0,350.0,263.0,397.0,312.0,343.0,343.0,254.0,144.0,119.0,92.0,2,=
7073,2022-09-30,ES59,269.0,194.0,201.0,187.0,187.0,189.0,181.0,296.0,232.0,193.0,125.0,114.0,1,=
7075,2022-09-30,ES59,198.0,193.0,169.0,202.0,248.0,249.0,230.0,230.0,168.0,128.0,79.0,74.0,2,=
7077,2022-09-30,ES60,472.0,543.0,489.0,488.0,618.0,577.0,565.0,532.0,348.0,210.0,161.0,162.0,1,=


In [ ]:
data_loader.data['datasets/estaciones'] = df
data_loader.data['datasets/estaciones']

,FDIA,FEST,HOR1,HOR2,HOR3,HOR4,HOR5,HOR6,HOR7,HOR8,HOR9,HOR10,HOR11,HOR12,FSEN_DIRECTION,FSEN_AMPM
0,2023-04-01,ES01,819.0,572.0,467.0,397.0,373.0,393.0,352.0,470.0,713.0,611.0,800.0,992.0,1,-
1,2023-04-01,ES01,1223.0,1165.0,832.0,884.0,1016.0,1058.0,1104.0,1407.0,1276.0,994.0,986.0,965.0,1,=
2,2023-04-01,ES01,683.0,464.0,361.0,298.0,297.0,416.0,326.0,338.0,526.0,765.0,993.0,1083.0,2,-
3,2023-04-01,ES01,1206.0,1077.0,748.0,923.0,964.0,1104.0,1159.0,1083.0,971.0,804.0,767.0,709.0,2,=
4,2023-04-01,ES02,312.0,309.0,262.0,213.0,187.0,129.0,150.0,170.0,193.0,260.0,315.0,433.0,1,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,2022-09-30,ES59,198.0,193.0,169.0,202.0,248.0,249.0,230.0,230.0,168.0,128.0,79.0,74.0,2,=
7076,2022-09-30,ES60,48.0,35.0,28.0,34.0,74.0,166.0,605.0,846.0,666.0,508.0,486.0,554.0,1,-
7077,2022-09-30,ES60,472.0,543.0,489.0,488.0,618.0,577.0,565.0,532.0,348.0,210.0,161.0,162.0,1,=
7078,2022-09-30,ES60,57.0,51.0,26.0,19.0,48.0,117.0,258.0,616.0,492.0,439.0,489.0,541.0,2,-


In [ ]:
np.ma.masked_where(df.FSEN_AMPM == '=', df.FSEN_AMPM, copy=False)

masked_array(data=['-', --, '-', ..., --, '-', --],
             mask=[False,  True, False, ...,  True, False,  True],
       fill_value='?',
            dtype=object)

In [ ]:
data_loader.dataframe_summary('datasets/estaciones')


ValueError: could not convert string to float: 'ES01'

In [ ]:
# Group by FEST, FDIA, and FSEN_number (first character of FSEN)
grouped = df[].groupby(['FEST', 'FDIA', df[pm_mask]['FSEN'].str[0]])

# Get the indices of rows that satisfy the merging condition
merge_indices = grouped.apply(lambda x: len(x) > 1).reset_index().loc[:, ['level_0']].values.flatten()

# Update new columns based on conditions using vectorized operations
fest_values = df.loc[merge_indices, 'FEST']
fdia_values = df.loc[merge_indices, 'FDIA']
fsen_numbers = df.loc[merge_indices, 'FSEN'].str[0]
fsen_hours = df.loc[merge_indices, 'FSEN_hour']

# Update the new columns based on the conditions using vectorized operations
df.loc[mask, f'HOR{df["FSEN_hour_bool"].astype(int) + 12}'] = df.loc[mask, 'FEST']

hor_columns = [f'HOR{fsen_hours + 12}']
df.loc[merge_indices.repeat(len(hor_columns)), hor_columns * len(merge_indices)] = fest_values.repeat(len(hor_columns))


# Drop unnecessary columns if needed
df.drop(['FEST', 'FDIA'], axis=1, inplace=True)

In [ ]:
data_loader.get_info('DireccionesEvolucionHistorica_20231004.csv')

Como podemos observar, faltan datos de las columnas VIA_SQC (secuencia de la denominación), con 371584 NaNs; VIA_PAR (partícula de la denominación), con 17997; CALIFICADOR (del número de la policía), con 322826; FECHA_DE_BAJA (de la dirección), con 211657; y TIPO_NDP (tipo de número: portal, garaje, fachada, etc.) con 16248.

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']

La secuencia de denominación (VIA_SQC) es solo NaNs, por lo que lo más lógico es eliminar la columna directamente

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv'].drop('VIA_SQC', axis=1)

Ahora, pasamos a la partícula de denominación (VIA_PAR), hay 7 valores distintos de entre la lista posible de 20 (DE, EL, A, DEL, LA, AL, DE LA, LAS, A LA, DE LAS, LO, A LAS, DE LO, LOS, A LO, DE LO, DE LOS, POR EL, A LOS, POR LA), de los cuales el primero es A LA y el último es DEL. Por lo tanto, parece que no se ha rellenado con Null o espacio vacío aquellas direcciones en las que no hay partícula:

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['VIA_PAR'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['VIA_PAR'].fillna(' ')

Fecha de baja, si no hay valor, es que está aún de alta, así que también sustituimos por un valor que represente que sigue dado de alta:

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'].fillna('--/--/----')

Para Tipo de número de policía, rellenaremos con el dato más habitual: PORTAL

In [ ]:
tipoNDP = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'].mode().iat[0]
data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv']['TIPO_NDP'].fillna(tipoNDP)

Para Calificador, como no sabemos realmente cuál es el criterio y la mayoría de datos son faltantes (322826 faltan de 371584), lo más prudente es eliminar la columna directamente

In [ ]:
data_loader.data['DireccionesEvolucionHistorica_20231004.csv'] = data_loader.data['DireccionesEvolucionHistorica_20231004.csv'].drop('CALIFICADOR', axis=1)

In [ ]:
data_loader.get_info('DireccionesEvolucionHistorica_20231004.csv')

Efectivamente no quedan Missing Values. Pasamos a las direcciones vigentes:

In [ ]:
data_loader.get_info('DireccionesVigentes_20231004.csv')

Faltan valores en Vía Par (3381) y en Calificador (178838), los sustituimos con el mismo criterio que en el dataset anterior:

In [ ]:
data_loader.data['DireccionesVigentes_20231004.csv']['VIA_PAR'] = data_loader.data['DireccionesVigentes_20231004.csv']['VIA_PAR'].fillna(' ')

In [ ]:
data_loader.data['DireccionesVigentes_20231004.csv'] = data_loader.data['DireccionesVigentes_20231004.csv'].drop('CALIFICADOR', axis=1)

In [ ]:
data_loader.get_info('DireccionesVigentes_20231004.csv')

Pasamos al dataset estrella: radares.

In [ ]:
data_loader.get_info('RADARES_FIJOS_vDTT.csv')
data_loader.data['RADARES_FIJOS_vDTT.csv']

Carretera o vial, PK, sentido y tipo tienen muy poca frecuencia de NaNs, por lo que solo sustituimos por los valores más frecuentes

In [ ]:
carretera = data_loader.data['RADARES_FIJOS_vDTT.csv']['CARRETARA_O_VIAL'].mode().iat[0]
pk = data_loader.data['RADARES_FIJOS_vDTT.csv']['PK'].mode().iat[0]
sentido = data_loader.data['RADARES_FIJOS_vDTT.csv']['SENTIDO'].mode().iat[0]
tipo = data_loader.data['RADARES_FIJOS_vDTT.csv']['TIPO'].mode().iat[0]

data_loader.data['RADARES_FIJOS_vDTT.csv']['CARRETARA_O_VIAL'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['CARRETARA_O_VIAL'].fillna(carretera)
data_loader.data['RADARES_FIJOS_vDTT.csv']['PK'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['PK'].fillna(pk)
data_loader.data['RADARES_FIJOS_vDTT.csv']['SENTIDO'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['SENTIDO'].fillna(sentido)
data_loader.data['RADARES_FIJOS_vDTT.csv']['TIPO'] = data_loader.data['RADARES_FIJOS_vDTT.csv']['TIPO'].fillna(tipo)

In [ ]:
data_loader.get_info('RADARES_FIJOS_vDTT.csv')

A continuación identificamos los faltantes en la evolución histórica de viales:


In [ ]:
data_loader.get_info('VialesEvolucionHistorica_20231004.csv')
data_loader.data['VialesEvolucionHistorica_20231004.csv']

Faltan VIA_PAR y FECHA_DE_BAJA, que ya se han tratado en dataframes previos y manejaremos de la misma manera:

In [ ]:
data_loader.data['VialesEvolucionHistorica_20231004.csv']['VIA_PAR'] = data_loader.data['VialesEvolucionHistorica_20231004.csv']['VIA_PAR'].fillna(' ')
data_loader.data['VialesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'] = data_loader.data['VialesEvolucionHistorica_20231004.csv']['FECHA_DE_BAJA'].fillna('--/--/----')

In [ ]:
data_loader.get_info('VialesEvolucionHistorica_20231004.csv')

Pasamos al dataframe de viales vigentes distritos barrios:

In [ ]:
data_loader.get_info('VialesVigentesDistritosBarrios_20231004.csv')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']

VIA_PAR hacemos la misma solución que previamente:

In [ ]:
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['VIA_PAR'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['VIA_PAR'].fillna(' ')

PAR_MIN, PAR_MAX, IMPAR_MAX e IMPAR_MIN tienen un valor para los casos en los que no hay calificador: '-', y es lo que usaremos para sustituir los datos faltantes.

In [ ]:
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MAX'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MIN'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['IMPAR_MIN'].fillna('-')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MAX'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MIN'] = data_loader.data['VialesVigentesDistritosBarrios_20231004.csv']['PAR_MIN'].fillna('-')

Pasamos a viales vigentes distritos:

In [ ]:
data_loader.get_info('VialesVigentesDistritos_20231004.csv')
data_loader.data['VialesVigentesDistritos_20231004.csv']

In [ ]:
data_loader.data['VialesVigentesDistritos_20231004.csv']['VIA_PAR'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['VIA_PAR'].fillna(' ')
data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MAX'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MIN'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['IMPAR_MIN'].fillna('-')
data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MAX'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MAX'].fillna('-')
data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MIN'] = data_loader.data['VialesVigentesDistritos_20231004.csv']['PAR_MIN'].fillna('-')

Pasamos al último dataset: viales vigentes a 4/10/2023

In [ ]:
data_loader.get_info('VialesVigentes_20231004.csv')
data_loader.data['VialesVigentes_20231004.csv']

Particula termina y partícula comienza está estrechamente relacionada con vía par, por lo que la sustitución de valores es igual

In [ ]:
data_loader.data['VialesVigentes_20231004.csv']['VIA_PAR'] = data_loader.data['VialesVigentes_20231004.csv']['VIA_PAR'].fillna(' ')
data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_TERMINA'] = data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_TERMINA'].fillna(' ')
data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_COMIENZA'] = data_loader.data['VialesVigentes_20231004.csv']['PARTICULA_COMIENZA'].fillna(' ')


### Análisis Descriptivo de los Datos
A continuación, veremos, dataframe a dataframe, el análisis descriptivo


In [ ]:
for i in data.keys():
    print(i)

Empezando con el dataframe de actuaciones bomberos:

In [ ]:
data_loader.dataframe_summary('datasets/actuacionesBomberos')


Tabla de frecuencia relativa de accidentes por distritos. En esta tabla se obtiene la frecuencia de accidentes en todos los distritos de madrid, esto hace que se pueda obtener también el tráfico de coches en el distrito. Aunque no sea una relación proporcional, a mayor número de accidentes supone un tráfico mayor en la zona.

In [ ]:
data_loader.data['datasets/accidentalidad']['DISTRITO'] = data_loader.data['datasets/accidentalidad']['DISTRITO'].astype(str)
CSVDataLoader.create_graph(data_loader.data['datasets/accidentalidad'], 'DISTRITO', 'accidentes por distrito')


Tabla comparativa que muestra los accidentes donde la persona que lo ha padecido ha dado positivo en alcohol.

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
positivos = accidentes[accidentes['POSITIVA_ALCOHOL'] == 'S']
negativos = accidentes[accidentes['POSITIVA_ALCOHOL'] == 'N']
labels = ['Positivo en Alcohol', 'Negativo en Alcohol', 'No se realizó la prueba']
values = [len(positivos), len(negativos), len(accidentes) - len(positivos) - len(negativos)]
plt.bar(labels, values)
plt.xlabel('Resultado de Alcohol')
plt.ylabel('Número de Accidentes')
plt.title('Comparación de Accidentes con y sin Positivo en Alcohol')
plt.show()

Tabla comparativo donde la persona que ha padecido el accidente ha dado positivo en drogas.

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
positivos = accidentes[accidentes['POSITIVA_DROGA'] == 1]
negativos = accidentes[accidentes['POSITIVA_DROGA'] == 0]
labels = ['Positivo en Drogas', 'Negativo en Drogas']
values = [len(positivos), len(negativos)]
plt.bar(labels, values)
plt.xlabel('Resultado de Drogas')
plt.ylabel('Número de Accidentes')
plt.title('Comparación de Accidentes con y sin Positivo en Drogas')
plt.show()

Tabla comparativa que relaciona los accidente en los que la persona ha dado positivo en alcohol y drogas

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
positivos_alcohol = accidentes[accidentes['POSITIVA_ALCOHOL'] == 'S']
positivos_drogas = accidentes[accidentes['POSITIVA_DROGA'] == 1]
positivos_alcohol_drogas = positivos_alcohol[positivos_alcohol['POSITIVA_DROGA'] == 1]
positivos_alcohol_drogas = len(positivos_alcohol_drogas)
negativos = len(accidentes) - positivos_alcohol_drogas
labels = ['Positivo en Alcohol y Drogas', 'Negativo en Alcohol y Drogas']
values = [positivos_alcohol_drogas, negativos]
plt.bar(labels, values)
plt.xlabel('Resultado de Alcohol')
plt.ylabel('Número de Accidentes')
plt.title('Comparación de Accidentes con y sin Positivo en Alcohol')
plt.show()

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
labels = accidentes['TIPO_ACCIDENTE'].unique()
values = accidentes['TIPO_ACCIDENTE'].value_counts()
percentages = (values / len(accidentes)) * 100
plt.pie(values, shadow=True, startangle=90)
plt.title('Tipos de Accidentes')
legend_labels = [f'{label} ({percentage:.2f}%)' for label, percentage in zip(labels, percentages)]
plt.legend(labels=legend_labels, loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

Este gráfico muestra los distintos tipos de accidentes registrados. Se puede ver que la colisión frontal-lateral y el alcance son las mauores causas de accidentes.

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']
labels = accidentes['LESIVIDAD'].unique()
values = accidentes['LESIVIDAD'].value_counts()
percentages = (values / len(accidentes)) * 100
plt.pie(values, shadow=True, startangle=90)
plt.title('Lesividad')
legend_labels = [f'{label} ({percentage:.2f}%)' for label, percentage in zip(labels, percentages)]
plt.legend(labels=legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

Este gráfico muestra el tipo de lesividad del accidente. Se aprecia que 3 de cada 4 personas no necesitan asistencia sanitaria.